# 🧹 02_data_cleaning_FIXED – Correção do merge IBGE x CEPEA

## Objetivo desta etapa

Nesta etapa vamos:
- Carregar os dados brutos do IBGE (PAM) e os dados de preços do CEPEA já pré-processados.
- Garantir que os caminhos e arquivos estão corretos antes de aplicar qualquer transformação.
- Inspecionar rapidamente a estrutura (colunas e número de linhas) de cada fonte.

O foco aqui é preparar o ambiente de limpeza que vai gerar o dataset integrado **sem merge cartesiano**, resultando em **48 registros esperados** (2 municípios × 12 anos × 2 produtos).


In [3]:
# CÉLULA 1: SETUP E CARREGAMENTO DOS DADOS

import os
import pandas as pd
import numpy as np
from google.colab import drive
from IPython.display import display

# Montar Google Drive
drive.mount("/content/drive")

# Definir diretórios base
BASE_DIR = "/content/drive/MyDrive/projeto_produtividade"
RAW_DATA_DIR = os.path.join(BASE_DIR, "data", "raw")
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "data", "processed")

# Caminhos dos arquivos de entrada
path_ibge = os.path.join(RAW_DATA_DIR, "pam_permanente_vsf_raw.csv")
path_cepea = os.path.join(PROCESSED_DATA_DIR, "cepea_precos_medios_anuais.csv")

print("Caminhos definidos. Carregando arquivos...")

try:
    df_ibge_raw = pd.read_csv(path_ibge)
    df_precos = pd.read_csv(path_cepea)
    print("Arquivos CSV do IBGE e CEPEA carregados com sucesso!")
except FileNotFoundError as e:
    raise FileNotFoundError(f"Erro ao carregar arquivo. Verifique os caminhos. Erro: {e}")

# Amostras dos dados
print("\nAmostra IBGE (bruto):")
display(df_ibge_raw.head(5))

print("\nAmostra CEPEA (preços):")
display(df_precos.head(5))

# Shapes para conferência rápida
print("\nShapes dos dataframes:")
print(f"IBGE raw        : {df_ibge_raw.shape}")
print(f"Preços CEPEA    : {df_precos.shape}")

Mounted at /content/drive
Caminhos definidos. Carregando arquivos...
Arquivos CSV do IBGE e CEPEA carregados com sucesso!

Amostra IBGE (bruto):


,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável,Produto das lavouras permanentes (Código),Produto das lavouras permanentes
0,6,Município,1006,Hectares,4642,2611101,Petrolina (PE),2013,2013,216,Área colhida,2748,Uva
1,6,Município,1006,Hectares,7880,2611101,Petrolina (PE),2013,2013,216,Área colhida,2737,Manga
2,6,Município,1017,Toneladas,162448,2611101,Petrolina (PE),2013,2013,214,Quantidade produzida,2748,Uva
3,6,Município,1017,Toneladas,173360,2611101,Petrolina (PE),2013,2013,214,Quantidade produzida,2737,Manga
4,6,Município,33,Quilogramas por Hectare,34995,2611101,Petrolina (PE),2013,2013,112,Rendimento médio da produção,2748,Uva



Amostra CEPEA (preços):


,ano,preco_r$_kg,produto
0,2024.0,13.71,Uva
1,2024.0,7.80,Uva
2,2024.0,15.42,Uva
3,2024.0,7.63,Uva
4,2024.0,11.84,Uva



Shapes dos dataframes:
IBGE raw        : (144, 13)
Preços CEPEA    : (153, 3)


## 🧮 Passo 2 – Padronizar IBGE e agregar preços CEPEA

Neste passo vamos:

- Padronizar as colunas fundamentais do dataset do IBGE (`df_ibge_raw`), renomeando para um padrão mais simples (`municipio`, `ano`, `produto`, `variavel`, `valor`).
- Filtrar apenas as variáveis de interesse: **Área colhida**, **Quantidade produzida** e **Rendimento médio da produção**.
- Filtrar apenas os produtos alvo: **Manga** e **Uva**.
- Agregar os preços do CEPEA (`df_precos`) por **ano + produto**, calculando:
  - mediana (mais robusta a outliers),
  - média,
  - desvio padrão,
  - número de observações de preço.

No final, teremos:
- Um `df_ibge` já limpo e focado nas métricas relevantes.
- Um `df_precos_agg` com **uma linha por ano-produto**, pronto para o merge sem multiplicação cartesiana.


In [4]:
# CÉLULA 2: PADRONIZAR IBGE E AGREGAR PREÇOS CEPEA

# 2.1 Padronizar colunas fundamentais do IBGE
df_ibge = df_ibge_raw.rename(columns={
    "Município": "municipio",
    "Ano": "ano",
    "Produto das lavouras permanentes": "produto",
    "Variável": "variavel",
    "Valor": "valor"
}).copy()

# Filtrar apenas variáveis de interesse
variaveis_alvo = [
    "Área colhida",
    "Quantidade produzida",
    "Rendimento médio da produção"
]

df_ibge = df_ibge[df_ibge["variavel"].isin(variaveis_alvo)]

# Filtrar apenas produtos de interesse (Manga e Uva)
produtos_alvo = ["Manga", "Uva"]
df_ibge = df_ibge[df_ibge["produto"].isin(produtos_alvo)]

print("Resumo IBGE após padronização e filtro:")
display(df_ibge[["municipio", "ano", "produto", "variavel", "valor"]].head())

print("\nShape IBGE filtrado:", df_ibge.shape)

# 2.2 Agregar preços CEPEA por ano-produto (mediana, média, std, contagem)
agg_dict = {"preco_r$_kg": ["median", "mean", "std", "count"]}

df_precos_agg = df_precos.groupby(["ano", "produto"], as_index=False).agg(agg_dict)

# Ajustar nomes das colunas após o groupby com múltiplas agregações
df_precos_agg.columns = [
    "ano",
    "produto",
    "preco_mediano_r$_kg",
    "preco_medio_r$_kg",
    "preco_std_r$_kg",
    "num_observacoes_preco"
]

print("\nResumo CEPEA após agregação:")
display(df_precos_agg.head())

print("\nShape CEPEA agregado:", df_precos_agg.shape)

# Checagem rápida de consistência entre bases
print("\nAnos IBGE:", sorted(df_ibge["ano"].unique()))
print("Anos CEPEA agregados:", sorted(df_precos_agg["ano"].unique()))
print("Produtos IBGE:", df_ibge["produto"].unique())
print("Produtos CEPEA agregados:", df_precos_agg["produto"].unique())


Resumo IBGE após padronização e filtro:


,municipio,ano,produto,variavel,valor
0,Petrolina (PE),2013,Uva,Área colhida,4642
1,Petrolina (PE),2013,Manga,Área colhida,7880
2,Petrolina (PE),2013,Uva,Quantidade produzida,162448
3,Petrolina (PE),2013,Manga,Quantidade produzida,173360
4,Petrolina (PE),2013,Uva,Rendimento médio da produção,34995



Shape IBGE filtrado: (144, 13)

Resumo CEPEA após agregação:


,ano,produto,preco_mediano_r$_kg,preco_medio_r$_kg,preco_std_r$_kg,num_observacoes_preco
0,2013.0,Manga,1.365,1.365000,0.417193,2
1,2013.0,Uva,4.300,4.443571,1.355959,14
2,2014.0,Manga,1.255,1.445000,0.610710,4
3,2014.0,Uva,4.335,4.574167,1.493017,12
4,2015.0,Manga,1.590,1.706667,0.642988,3



Shape CEPEA agregado: (24, 6)

Anos IBGE: [np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
Anos CEPEA agregados: [np.float64(2013.0), np.float64(2014.0), np.float64(2015.0), np.float64(2016.0), np.float64(2017.0), np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0), np.float64(2024.0)]
Produtos IBGE: ['Uva' 'Manga']
Produtos CEPEA agregados: ['Manga' 'Uva']


## 🔄 Passo 3 – Pivot do IBGE e merge com preços (sem cartesiano)

Neste passo vamos:

- Transformar o dataframe do IBGE (`df_ibge`) de formato **longo** (uma linha por variável) para formato **wide** (uma linha por `municipio-ano-produto`), criando colunas específicas para:
  - `area_colhida_ha`
  - `quantidade_produzida_t`
  - `rendimento_medio_kg_ha`
- Garantir que as chaves de merge são **únicas**:
  - `df_precos_agg`: uma linha por `ano-produto`.
  - `df_ibge_wide`: uma linha por `municipio-ano-produto`.
- Realizar o merge entre IBGE e CEPEA usando `merge(..., validate="m:1")` para impedir multiplicação cartesiana.

No final, teremos um `df_merged` com o número correto de linhas (combinações de município × produto × ano) e colunas de área, produção, rendimento e preços agregados, pronto para ser salvo como dataset integrado.


In [5]:
# CÉLULA 3: PIVOTAR IBGE PARA FORMATO WIDE E FAZER MERGE COM PREÇOS

# 3.1 Mapear descrições das variáveis do IBGE para nomes de colunas mais amigáveis
pivot_map = {
    "Área colhida": "area_colhida_ha",
    "Quantidade produzida": "quantidade_produzida_t",
    "Rendimento médio da produção": "rendimento_medio_kg_ha",
}

df_ibge["variavel_simplificada"] = df_ibge["variavel"].map(pivot_map)

# Checagem de segurança: ver se alguma variável ficou sem mapeamento
if df_ibge["variavel_simplificada"].isna().any():
    print("ATENÇÃO: Existem variáveis do IBGE sem mapeamento em pivot_map:")
    display(
        df_ibge[df_ibge["variavel_simplificada"].isna()][["variavel"]]
        .drop_duplicates()
    )

# 3.2 Pivotar IBGE para uma linha por municipio-ano-produto
df_ibge_wide = (
    df_ibge
    .pivot_table(
        index=["municipio", "ano", "produto"],
        columns="variavel_simplificada",
        values="valor",
        aggfunc="first"  # não deve haver duplicidade por chave
    )
    .reset_index()
)

# Organizar colunas (chaves primeiro)
cols_base = ["municipio", "ano", "produto"]
other_cols = [c for c in df_ibge_wide.columns if c not in cols_base]
df_ibge_wide = df_ibge_wide[cols_base + other_cols]

print("IBGE em formato wide (uma linha por municipio-ano-produto):")
display(df_ibge_wide.head())

print("\nShape IBGE wide:", df_ibge_wide.shape)

# 3.3 Checar unicidade das chaves antes do merge

print("\nChecando unicidade de chaves (ano, produto) em CEPEA agregado:")
dup_keys_precos = (
    df_precos_agg
    .groupby(["ano", "produto"])
    .size()
    .reset_index(name="n")
)
display(dup_keys_precos[dup_keys_precos["n"] > 1])

print("\nChecando unicidade de chaves (municipio, ano, produto) em IBGE wide:")
dup_keys_ibge = (
    df_ibge_wide
    .groupby(["municipio", "ano", "produto"])
    .size()
    .reset_index(name="n")
)
display(dup_keys_ibge[dup_keys_ibge["n"] > 1])

# 3.4 Merge seguro (sem multiplicação cartesiana)
df_merged = pd.merge(
    df_ibge_wide,
    df_precos_agg,
    on=["ano", "produto"],
    how="left",
    validate="m:1"  # cada linha de IBGE casa com no máximo 1 linha de preço
)

print("\nShape final após merge:", df_merged.shape)
print("Preview do dataset integrado (IBGE + preços CEPEA):")
display(df_merged.head())

# 3.5 Checagem final de número de linhas
print(
    "\nNúmero de linhas em IBGE wide (esperado para o dataset final):",
    df_ibge_wide.shape[0]
)

IBGE em formato wide (uma linha por municipio-ano-produto):


variavel_simplificada,municipio,ano,produto,area_colhida_ha,quantidade_produzida_t,rendimento_medio_kg_ha
0,Juazeiro (BA),2013,Manga,8086,210236,26000
1,Juazeiro (BA),2013,Uva,1270,27940,22000
2,Juazeiro (BA),2014,Manga,2130,55380,26000
3,Juazeiro (BA),2014,Uva,1576,39400,25000
4,Juazeiro (BA),2015,Manga,2130,44730,21000



Shape IBGE wide: (48, 6)

Checando unicidade de chaves (ano, produto) em CEPEA agregado:


,ano,produto,n



Checando unicidade de chaves (municipio, ano, produto) em IBGE wide:


,municipio,ano,produto,n



Shape final após merge: (48, 10)
Preview do dataset integrado (IBGE + preços CEPEA):


,municipio,ano,produto,area_colhida_ha,quantidade_produzida_t,rendimento_medio_kg_ha,preco_mediano_r$_kg,preco_medio_r$_kg,preco_std_r$_kg,num_observacoes_preco
0,Juazeiro (BA),2013,Manga,8086,210236,26000,1.365,1.365000,0.417193,2
1,Juazeiro (BA),2013,Uva,1270,27940,22000,4.300,4.443571,1.355959,14
2,Juazeiro (BA),2014,Manga,2130,55380,26000,1.255,1.445000,0.610710,4
3,Juazeiro (BA),2014,Uva,1576,39400,25000,4.335,4.574167,1.493017,12
4,Juazeiro (BA),2015,Manga,2130,44730,21000,1.590,1.706667,0.642988,3



Número de linhas em IBGE wide (esperado para o dataset final): 48


## 💾 Passo 4 – Validação final e salvamento do dataset integrado

Nesta etapa vamos:
- Verificar se o dataset integrado (`df_merged`) tem exatamente **48 registros**, conforme o plano (2 municípios × 12 anos × 2 produtos). [attached_file:1]
- Garantir que não existam duplicatas na chave primária (`municipio`, `ano`, `produto`). [attached_file:1]
- Checar a estrutura básica (número de municípios, intervalo de anos e produtos). [attached_file:1]
- Salvar o dataset corrigido em `PROCESSED_DATA_DIR` com o nome `pam_censo_agro_integrado_v2.csv`, que será o novo input do notebook `03_exploratory_data_analysis.ipynb`. [attached_file:1]


In [6]:
# CÉLULA 4: VALIDAÇÃO FINAL E SALVAMENTO DO DATASET INTEGRADO

# 4.1 Validação do número de linhas
expected_rows = 2 * 12 * 2  # 2 municípios × 12 anos (2013-2024) × 2 produtos
n_rows = df_merged.shape[0]

print(f"Total de linhas em df_merged: {n_rows}")
assert n_rows == expected_rows, f"❌ Esperado {expected_rows}, obtido {n_rows}"

# 4.2 Verificar duplicatas na chave primária
duplicates = df_merged.duplicated(subset=["municipio", "ano", "produto"]).sum()
print(f"Total de duplicatas na chave [municipio, ano, produto]: {duplicates}")
assert duplicates == 0, f"❌ Encontradas {duplicates} duplicatas na chave!"

# 4.3 Estrutura básica do dataset
print("\n📊 Estrutura final do dataset integrado:")
print(f"- Municípios: {df_merged['municipio'].nunique()}")
print(f"- Anos      : {df_merged['ano'].min()} - {df_merged['ano'].max()}")
print(f"- Produtos  : {df_merged['produto'].unique()}")
print(f"- Registros : {len(df_merged)}")

# 4.4 Salvar CSV no diretório processed
output_path = os.path.join(PROCESSED_DATA_DIR, "pam_censo_agro_integrado_v2.csv")
df_merged.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"\n✅ Dataset salvo em: {output_path}")

Total de linhas em df_merged: 48
Total de duplicatas na chave [municipio, ano, produto]: 0

📊 Estrutura final do dataset integrado:
- Municípios: 2
- Anos      : 2013 - 2024
- Produtos  : ['Manga' 'Uva']
- Registros : 48

✅ Dataset salvo em: /content/drive/MyDrive/projeto_produtividade/data/processed/pam_censo_agro_integrado_v2.csv
